In [2]:
library('fastDummies')
library('pmml')
library('missForest')

Loading required package: XML

Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: foreach

Loading required package: itertools

Loading required package: iterators



In [3]:
data <- read.csv('./hmeq.csv', header=T, na.string=c(""))

In [4]:
ncol(data)

[1] 13

In [5]:
head(data)

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>
1,1,1100,25860,39025,HomeImp,Other,10.5,0,0,94.36667,1,9,NA
2,1,1300,70053,68400,HomeImp,Other,7.0,0,2,121.83333,0,14,NA
3,1,1500,13500,16700,HomeImp,Other,4.0,0,0,149.46667,1,10,NA
4,1,1500,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,0,1700,97800,112000,HomeImp,Office,3.0,0,0,93.33333,0,14,NA
6,1,1700,30548,40320,HomeImp,Other,9.0,0,0,101.46600,1,8,37.11361


In [6]:
data <- data.frame( CLAGE = as.numeric(data$CLAGE),
                 CLNO = as.numeric(data$CLNO),
                 DEBTINC = as.numeric(data$DEBTINC),
                 DELINQ = as.numeric(data$DELINQ),
                 DEROG = as.numeric(data$DEROG),
                 LOAN = as.numeric(data$LOAN),
                 MORTDUE = as.numeric(data$MORTDUE),
                 NINQ = as.numeric(data$NINQ),
                 VALUE = as.numeric(data$VALUE),
                 YOJ = as.numeric(data$YOJ),
                 JOB = as.factor(data$JOB),
                 REASON = as.factor(data$REASON),
                 BAD = as.factor(data$BAD),
                 stringsAsFactors = FALSE)

In [8]:
imp <- missForest(data, maxiter = 2, ntree = 10)

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!


In [7]:
imp$OOBerror

NRMSE       PFC 
0.1846278 0.1069916

In [9]:
imp_f <- imp$ximp

In [10]:
data_dummies <- dummy_cols(imp_f)
head(data_dummies)

,CLAGE,CLNO,DEBTINC,DELINQ,DEROG,LOAN,MORTDUE,NINQ,VALUE,YOJ,⋯,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,REASON_DebtCon,REASON_HomeImp,BAD_0,BAD_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,94.36667,9.0,35.07261,0,0.0,1100,25860.0,1.0,39025.00,10.5,⋯,0,0,1,0,0,0,0,1,0,1
2,121.83333,14.0,35.54922,2,0.0,1300,70053.0,0.0,68400.00,7.0,⋯,0,0,1,0,0,0,0,1,0,1
3,149.46667,10.0,29.73714,0,0.0,1500,13500.0,1.0,16700.00,4.0,⋯,0,0,1,0,0,0,0,1,0,1
4,133.71092,11.6,26.95384,1,0.5,1500,29522.3,0.9,43861.53,6.3,⋯,0,0,1,0,0,0,0,1,0,1
5,93.33333,14.0,31.37202,0,0.0,1700,97800.0,0.0,112000.00,3.0,⋯,0,1,0,0,0,0,0,1,1,0
6,101.46600,8.0,37.11361,0,0.0,1700,30548.0,1.0,40320.00,9.0,⋯,0,0,1,0,0,0,0,1,0,1


In [12]:
final_df <- within(data_dummies, rm("JOB", "REASON", "BAD_1", "BAD_0"))
final_df$BAD =data_dummies$BAD

In [14]:
head(final_df)

,CLAGE,CLNO,DEBTINC,DELINQ,DEROG,LOAN,MORTDUE,NINQ,VALUE,YOJ,BAD,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,REASON_DebtCon,REASON_HomeImp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,94.36667,9.0,35.07261,0,0.0,1100,25860.0,1.0,39025.00,10.5,1,0,0,1,0,0,0,0,1
2,121.83333,14.0,35.54922,2,0.0,1300,70053.0,0.0,68400.00,7.0,1,0,0,1,0,0,0,0,1
3,149.46667,10.0,29.73714,0,0.0,1500,13500.0,1.0,16700.00,4.0,1,0,0,1,0,0,0,0,1
4,133.71092,11.6,26.95384,1,0.5,1500,29522.3,0.9,43861.53,6.3,1,0,0,1,0,0,0,0,1
5,93.33333,14.0,31.37202,0,0.0,1700,97800.0,0.0,112000.00,3.0,0,0,1,0,0,0,0,0,1
6,101.46600,8.0,37.11361,0,0.0,1700,30548.0,1.0,40320.00,9.0,1,0,0,1,0,0,0,0,1


In [11]:
dm_modelformula

ERROR: Error in eval(expr, envir, enclos): object 'dm_modelformula' not found


In [22]:
model <- glm(formula = BAD ~ ., family = binomial(link = "logit"), data = final_df)

s <- summary(model)

In [23]:
s


Call:
glm(formula = BAD ~ ., family = binomial(link = "logit"), data = final_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3099  -0.5878  -0.3919  -0.1956   3.6782  

Coefficients: (2 not defined because of singularities)
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -4.279e+00  3.348e-01 -12.782  < 2e-16 ***
CLAGE          -2.983e-03  5.142e-04  -5.802 6.56e-09 ***
CLNO           -3.002e-02  4.554e-03  -6.594 4.29e-11 ***
DEBTINC         1.227e-01  6.819e-03  17.994  < 2e-16 ***
DELINQ          7.690e-01  3.994e-02  19.253  < 2e-16 ***
DEROG           5.035e-01  4.863e-02  10.353  < 2e-16 ***
LOAN           -2.281e-05  4.395e-06  -5.190 2.10e-07 ***
MORTDUE        -8.432e-06  1.908e-06  -4.418 9.95e-06 ***
NINQ            1.355e-01  2.132e-02   6.355 2.09e-10 ***
VALUE           5.367e-06  1.366e-06   3.930 8.51e-05 ***
YOJ            -9.658e-03  5.837e-03  -1.655  0.09801 .  
JOB_Mgr        -5.564e-01  2.359e-01  -2.359  0.0183

In [26]:
print(s)


Call:
glm(formula = BAD ~ ., family = binomial(link = "logit"), data = final_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3099  -0.5878  -0.3919  -0.1956   3.6782  

Coefficients: (2 not defined because of singularities)
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -4.279e+00  3.348e-01 -12.782  < 2e-16 ***
CLAGE          -2.983e-03  5.142e-04  -5.802 6.56e-09 ***
CLNO           -3.002e-02  4.554e-03  -6.594 4.29e-11 ***
DEBTINC         1.227e-01  6.819e-03  17.994  < 2e-16 ***
DELINQ          7.690e-01  3.994e-02  19.253  < 2e-16 ***
DEROG           5.035e-01  4.863e-02  10.353  < 2e-16 ***
LOAN           -2.281e-05  4.395e-06  -5.190 2.10e-07 ***
MORTDUE        -8.432e-06  1.908e-06  -4.418 9.95e-06 ***
NINQ            1.355e-01  2.132e-02   6.355 2.09e-10 ***
VALUE           5.367e-06  1.366e-06   3.930 8.51e-05 ***
YOJ            -9.658e-03  5.837e-03  -1.655  0.09801 .  
JOB_Mgr        -5.564e-01  2.359e-01  -2.359  0.0183

In [27]:
saveXML(pmml(model, model.name="General_Regression_Model",
             app.name="Rattle/PMML",
             description="Generalized Linear Regression Model"), "R_HMEQ.xml")

[1] "R_HMEQ.xml"

In [29]:
write.csv(final_df, './hmeq_dummies.csv', row.names=FALSE)